# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,1.38,72,19,17.40,TF,1729030681
1,1,blackmans bay,-43.0167,147.3167,12.63,62,95,3.13,AU,1729030682
2,2,happy valley-goose bay,53.3168,-60.3315,2.17,93,100,4.63,CA,1729030509
3,3,adamstown,-25.0660,-130.1015,21.25,85,92,7.32,PN,1729030684
4,4,czarna bialostocka,53.3051,23.2815,5.46,90,18,2.60,PL,1729030685


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles="OSM", c='City', size='Humidity', alpha=0.7, hover_cols=['City', 'Country']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
temp_in_kelvin = 15 + 273.15

filtered_cities_df = city_data_df[
    (city_data_df['Max Temp'] > temp_in_kelvin) & 
    (city_data_df['Cloudiness'] > 50) &    
    (city_data_df['Wind Speed'] > 5) &     
    (city_data_df['Humidity'] == 0)        
]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
print(filtered_cities_df.head())

Empty DataFrame
Columns: [City_ID, City, Lat, Lng, Max Temp, Humidity, Cloudiness, Wind Speed, Country, Date]
Index: []


In [5]:
# Count the number of cities in your DataFrame
num_cities = city_data_df['City'].nunique()
print(f"Number of unique cities in the DataFrame: {num_cities}")

Number of unique cities in the DataFrame: 574


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
print(hotel_df.head())

                     City Country      Lat       Lng  Humidity Hotel Name
0       port-aux-francais      TF -49.3500   70.2167        72           
1           blackmans bay      AU -43.0167  147.3167        62           
2  happy valley-goose bay      CA  53.3168  -60.3315        93           
3               adamstown      PN -25.0660 -130.1015        85           
4      czarna bialostocka      PL  53.3051   23.2815        90           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 1000  # Set the radius to 1000 meters
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
port-aux-francais - nearest hotel: Keravel
blackmans bay - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
czarna bialostocka - nearest hotel: No hotel found
qaqortoq - nearest hotel: Hotel Qaqortoq
palikir - national government center - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
nizhneudinsk - nearest hotel: Полюс
butha-buthe - nearest hotel: No hotel found
tawau - nearest hotel: Hotel Walaitokou
ulladulla - nearest hotel: No hotel found
sisimiut - nearest hotel: Hotel SØMA
staritsa - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
waitangi - nearest hotel: Hotel Chathams
jamestown - nearest hotel: DoubleTree Jamestown
khandyga - nearest hotel: No hotel found
ende - nearest hotel: LCR H

namtsy - nearest hotel: No hotel found
serowe - nearest hotel: No hotel found
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
hard - nearest hotel: Hotel am See
lyantonde - nearest hotel: No hotel found
okhotsk - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
homer - nearest hotel: Driftwood Inn & Homer Seaside Lodges
selfoss - nearest hotel: Miðgarður
trincomalee - nearest hotel: Summer set inn
el granada - nearest hotel: Beach House
yarmouth - nearest hotel: No hotel found
wongsorejo - nearest hotel: No hotel found
shalqar - nearest hotel: Бекнур
scarborough - nearest hotel: The Victoria Hotel
tagaw-bay - nearest hotel: No hotel found
tralee - nearest hotel: The Ashe Hotel
celestun - nearest hotel: Hotel Flamingo Playa
mersing - nearest hotel: Mersing Inn Hotel
richards bay - nearest hotel: No hotel found
ponta delgada - nearest hotel: Hotel Talisman
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
karratha - nearest hotel: Kar

chivhu - nearest hotel: Vic's Travern
vanimo - nearest hotel: CBC Guesthouse
sao joao da barra - nearest hotel: No hotel found
broome - nearest hotel: No hotel found
qingdao - nearest hotel: No hotel found
belel - nearest hotel: No hotel found
motygino - nearest hotel: No hotel found
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
nar'yan-mar - nearest hotel: No hotel found
karachi - nearest hotel: Quetta Agha Hotel
daru - nearest hotel: Kuki
shasta lake - nearest hotel: No hotel found
smithers - nearest hotel: Sunshine Inn Hotel
kas - nearest hotel: KEKOVA OTEL
kill devil hills - nearest hotel: Mariner Days Inn & Suites
pineda de mar - nearest hotel: Hotel Mercè
parabel' - nearest hotel: Берлога
leava - nearest hotel: Fia fia
conduaga - nearest hotel: No hotel found
dombarovskiy - nearest hotel: No hotel found
harstad - nearest hotel: Scandic Harstad
kilindoni - nearest hotel: Ibizza Inn
at taj - nearest hotel: No hotel found
leninskiy - nearest hotel: No hotel found
kamina - 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,72,Keravel
1,blackmans bay,AU,-43.0167,147.3167,62,No hotel found
2,happy valley-goose bay,CA,53.3168,-60.3315,93,No hotel found
3,adamstown,PN,-25.0660,-130.1015,85,No hotel found
4,czarna bialostocka,PL,53.3051,23.2815,90,No hotel found


In [19]:
print(hotel_df[['City', 'Hotel Name']])

                       City               Hotel Name
0         port-aux-francais                  Keravel
1             blackmans bay           No hotel found
2    happy valley-goose bay           No hotel found
3                 adamstown           No hotel found
4        czarna bialostocka           No hotel found
..                      ...                      ...
569                mil'kovo                   Дрозды
570       caleta de carquin         Hotel Mi Choacan
571                riohacha                 IMPERIAL
572                pimentel  Hospedaje Calle del Sol
573          hassi messaoud                      AMC

[574 rows x 2 columns]


In [20]:
city_data_df = city_data_df.drop(columns=['Hotel Name'], errors='ignore')
city_data_df = city_data_df.merge(hotel_df[['City', 'Hotel Name']], on='City', how='left')
print(city_data_df[['City', 'Hotel Name']].head())

                     City      Hotel Name
0       port-aux-francais         Keravel
1           blackmans bay  No hotel found
2  happy valley-goose bay  No hotel found
3               adamstown  No hotel found
4      czarna bialostocka  No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles="OSM", c='City', size='Humidity', alpha=0.7, hover_cols=['City', 'Country', 'Hotel Name', 'Humidity']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)